In [31]:
# | default_exp routes.pdp

In [1]:
# | export
import httpx
import pandas as pd
import io

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [2]:
# | export
class PDP_NotRetrieved(de.DomoError):
    def __init__(
        self,
        domo_instance,
        function_name,
        status,
        message,
        pdp_id=None,
    ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=pdp_id,
            function_name=function_name,
            status=status,
            message=message,
        )

In [3]:
# | export
# async def get_pdp_policies(
#     auth: dmda.DomoAuth,
#     dataset_id: str,
#     debug_api: bool = False,
# ) -> rgd.ResponseGetData:
#     url = f"http://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/"

#     if debug_api:
#         print(url)

#     # headers = {
#     #     'Accept':'application/json',
#     #     'x-domo-authentication': auth.token
#     # }
#     res = await gd.get_data(
#         auth=auth, 
#         url=url, 
#         method="GET", 
#         debug_api=debug_api, 
#         headers= {'accept': 'application/json'},
#         params = {'options':'load_associations,load_filters,include_open_policy'}
#     )

#     if len(res.response) == 0 or not res.is_success:
#         raise PDP_NotRetrieved(
#             domo_instance=auth.domo_instance,
#             function_name="get_pdp_policies",
#             status=res.status,
#             message="failed to retrieve pdp policies",
#         )

#     return res

### Sample implementation for get_pdp_policies

In [5]:
# import os

# token_auth = dmda.DomoTokenAuth(
#     domo_instance="domo-community",
#     domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
# )

# # try:
# #     full_auth = dmda.DomoFullAuth(domo_instance="domo-community",
# #                              domo_username=os.environ["DOMO_USERNAME"],
# #                              domo_password=os.environ["DOJO_PASSWORD"]
# #                              )
# #     token = await full_auth.get_auth_token()
# # except dmda.InvalidCredentialsError as e:
# #     print(e)

# dataset_id = os.environ["DOJO_DATASET_ID"]

# try:
#     res = await get_pdp_policies(auth=token_auth, dataset_id=dataset_id, debug_api=True)
#     print(res.response)
# except PDP_NotRetrieved as e:
#     print(e)


http://domo-community.domo.com/api/query/v1/data-control/0e7f7755-eccc-4612-890c-c3dc3d5002c3/filter-groups/
🐛 debugging get_data
{'body': None,
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json',
             'x-domo-developer-token': '2f4cb212853732ef796abd46077cf1162c0a3ed7cf1ff079'},
 'method': 'GET',
 'params': {'options': 'load_associations,load_filters,include_open_policy'},
 'url': 'http://domo-community.domo.com/api/query/v1/data-control/0e7f7755-eccc-4612-890c-c3dc3d5002c3/filter-groups/'}
get_data: no body
get_data_response <Response [301 Moved Permanently]>
<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>nginx</center>
</body>
</html>



In [5]:
# SCRIPT FROM PA -- PULLS PDP POLICIES FROM DATASET

# import requests
# import json

# def getPDPList(instance, datasetId, token):
#     url = f'http://{instance}/api/query/v1/data-control/{datasetId}/filter-groups/' 
#     headers = {
#         'Accept':'application/json',
#         'Content-Type':'application/json',
#         'x-domo-authentication': token
#     }
#     response = requests.get(url, headers = headers, params = {'options':'load_associations,load_filters,include_open_policy'})
#     jsonResponse = json.loads(response.text)
#     return jsonResponse



In [6]:
# SAMPLE IMPLEMENTATION OF PA SCRIPT
# import os 

# full_auth = dmda.DomoFullAuth(domo_instance="domo-community",
#                              domo_username=os.environ["DOMO_USERNAME"],
#                              domo_password=os.environ["DOJO_PASSWORD"]
#                              )
# token = await full_auth.get_auth_token()
# res = getPDPList(instance='domo-community.domo.com', datasetId=os.environ["DOJO_DATASET_ID"], token=full_auth.token)
# res

[{'name': 'All Rows',
  'filterGroupId': 1152,
  'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3',
  'type': 'open',
  'dataSourcePermissions': False},
 {'name': 'Test Policy Name Employee 123',
  'filterGroupId': 1153,
  'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3',
  'type': 'user',
  'dataSourcePermissions': False,
  'parameters': [{'name': 'Employee ID',
    'value': '456',
    'values': ['456'],
    'type': 'COLUMN',
    'operator': 'EQUALS',
    'not': False,
    'ignoreCase': True}]},
 {'name': 'Test 456',
  'filterGroupId': 1156,
  'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3',
  'type': 'user',
  'userIds': [2072616249],
  'dataSourcePermissions': False,
  'resources': {'USER': ['2072616249']},
  'parameters': [{'name': 'Employee ID',
    'value': '465',
    'values': ['465'],
    'type': 'COLUMN',
    'operator': 'EQUALS',
    'not': False,
    'ignoreCase': False}]}]

In [6]:
class SearchPDP_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name = "search_pdp_by_name"):
        super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)
        
def search_pdp_policies_by_name(
        # used to return pdp policy info, search by name
        search_name: str,
        res: list[dict], # this is the res.response from get_pdp_policies -- should be list of dict
        ):
    
    match_group = next((policy for policy in res if policy.get('name') == search_name), None)
    #print(match_group)
    
    if not match_group:
        raise SearchPDP_Error(
            status='',
            message=f'There is no policy named {search_name}',
            domo_instance=''
        )
    res = match_group
    
    return res

### Sample implementation of search_pdp_policies_by_name

In [7]:
pdp_policy_search = search_pdp_policies_by_name(search_name='Test 456', res=res)
pdp_policy_search

TypeError: 'ResponseGetData' object is not iterable

## generate body for create_policy endpoint


In [21]:
# | export
def generate_policy_parameter_simple(
    column_name: str,
    column_values_ls: list[str] = None,
    operator="EQUALS",
    ignore_case: bool = True,
):
    if not isinstance(column_values_ls, list):
        column_values_ls = [column_values_ls]

    return {
        "type": "COLUMN",
        "name": column_name,
        "values": column_values_ls,
        "operator": operator,
        "ignoreCase": ignore_case,
    }

In [22]:
# | export
def generate_policy_body(
    policy_name: str,
    dataset_id: str,
    parameters_ls: dict,  # generated by generate_policy_parameter_simple method
    policy_id: str = None,  # only included if updating existing policy
    user_ids: list[str] = None,
    group_ids: list[str] = None,
    virtual_user_ids: list[str] = None,
):
    if not user_ids:
        user_ids = []

    if not group_ids:
        group_ids = []

    if not virtual_user_ids:
        virtual_user_ids = []

    if not isinstance(parameters_ls, list):
        parameters_ls = [parameters_ls]


    body = {
        "name": policy_name,
        "dataSourceId": dataset_id,
        "userIds": user_ids,
        "virtualUserIds": virtual_user_ids,
        "groupIds": group_ids,
        "dataSourcePermissions": False,
        "parameters": parameters_ls,
    }

    if policy_id:
        body.update({"filterGroupId": policy_id})

    return body

## Policy CRUD Operations

In [40]:
# | export
async def create_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups"

    if debug_api:
        print(url)
    
    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of create_policy, generate_policy_parameter_simple and generate_policy_body

In [41]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameter = generate_policy_parameter_simple(
    column_name="Employee ID",
    column_values_ls=['123'],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    "Test Policy Name Employee 123",
    dataset_id = dataset_id,
    policy_id="123",  # including the policy_id updates that existing policy
    parameters_ls=sample_parameter,
    user_ids=['1893952720']
)

res = await create_policy(
    auth = token_auth,
    dataset_id = dataset_id,
    body = body,
)
res.response

{'name': 'Test Policy Name Employee 123',
 'filterGroupId': 1155,
 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3',
 'policySetId': 1130,
 'userIds': [1893952720],
 'dataSourcePermissions': False,
 'parameters': [{'name': 'Employee ID',
   'value': '123',
   'values': ['123'],
   'type': 'COLUMN',
   'operator': 'EQUALS',
   'not': False,
   'ignoreCase': True}]}

In [42]:
# | export
async def update_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    policy_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/{policy_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of update_policy

In [54]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameters = generate_policy_parameter_simple(
    column_name="Employee ID",
    column_values_ls=["456"],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    "Test Policy Name Employee 123",
    dataset_id=dataset_id,
    # policy_id='1153',
    parameters_ls=[sample_parameters],
    user_ids=['1893952720']
)

res = await update_policy(
    auth=token_auth,
    dataset_id=dataset_id,
    policy_id='1153',
    body=body,
)

res

ResponseGetData(status=200, response={'name': 'Test Policy Name Employee 123', 'filterGroupId': 1153, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'dataSourcePermissions': False, 'parameters': [{'name': 'Employee ID', 'value': '456', 'values': ['456'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': True}]}, is_success=True)

In [55]:
# | export

async def toggle_pdp(
        auth: dmda.DomoAuth,
        dataset_id: str,
        is_enable: bool = True,
        debug_api: bool = False,
        session: httpx.AsyncClient = None
    )-> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}"

    if debug_api:
        print(url)

    body = {"enabled": is_enable,
            "external": False  # not sure what this parameter does
            }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res


#### sample implementation of toggle_pdp

In [57]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

res = await toggle_pdp(
    auth=token_auth,
    dataset_id=dataset_id,
    is_enable = True,
)
res

ResponseGetData(status=200, response={'enabled': True, 'secured': False, 'external': False}, is_success=True)

In [46]:
# | hide

import nbdev

nbdev.nbdev_export()